In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import nibabel as nib
import pandas as pd
import numpy as np
from torchvision import transforms
from models.resnet_alzheimer import resnet18
import torchio as tio
from tqdm import tqdm

#### **1. ResNet18:**

**ResNet18** es una arquitectura de red neuronal convolucional muy popular, parte de la familia Residual Networks (ResNet). Su innovación principal radica en el uso de **"bloques residuales"** o "saltos" (skip connections). Estos saltos permiten que la información y los gradientes fluyan más fácilmente a través de muchas capas, resolviendo el problema del "gradiente desvanecido" que dificultaba el entrenamiento de redes muy profundas.

* **Pre-entrenamiento:** Comúnmente, ResNet18 se pre-entrena en el dataset **ImageNet**. Este es un gigantesco conjunto de datos con millones de imágenes de objetos cotidianos (perros, coches, sillas, etc.) de 1000 categorías diferentes.
* **Ventajas en imágenes médicas:** A pesar de haber sido entrenado con imágenes naturales, las características de bajo nivel que ResNet18 aprende de ImageNet (detección de bordes, patrones de textura) son sorprendentemente útiles como punto de partida para el análisis de imágenes médicas. Es un excelente punto de inicio general.
* **Consideración:** Puede haber una "brecha de dominio" entre las imágenes de ImageNet y las imágenes médicas, lo que significa que el fine-tuning es crucial para adaptar el modelo a las particularidades de los datos médicos (contrastes, resoluciones, tipos de ruido).

---

#### **2. MedicalNet:**

**MedicalNet** es una iniciativa que proporciona modelos (incluyendo variantes de ResNet como ResNet18) que han sido **pre-entrenados específicamente en un vasto y diverso conjunto de datos de imágenes médicas**.

* **Pre-entrenamiento:** A diferencia de ResNet18 estándar, MedicalNet ha sido entrenado con millones de imágenes provenientes de diversas modalidades médicas (MRI, CT, rayos X, ultrasonido) y cubriendo diferentes órganos y patologías.
* **Ventajas en imágenes médicas:**
    * **Mayor relevancia de las características:** Al estar pre-entrenado en datos médicos, MedicalNet ya ha aprendido patrones y características que son intrínsecamente más relevantes para el diagnóstico y análisis clínico.
    * **Menor brecha de dominio:** Esto puede traducirse en un mejor rendimiento inicial, una convergencia más rápida durante el fine-tuning y, potencialmente, un mejor rendimiento final con menos datos de entrenamiento específicos para tu tarea.
    * **Adaptado a diferentes modalidades:** Su entrenamiento diverso lo hace robusto para trabajar con distintos tipos de imágenes médicas.



Pesos de MedicalNet y ResNet18:
https://share.weiyun.com/55sZyIx 

In [19]:
class AlzheimerDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx]['ID']
        label = self.data.iloc[idx]['CDR']

        img = nib.load(img_path).get_fdata()
        img = torch.tensor(img, dtype=torch.float32).unsqueeze(0)  # Add channel dimension
        if self.transform:
            img = self.transform(img)
        return img, torch.tensor(label, dtype=torch.float32)


def custom_collate(batch):
    batch = [item for item in batch if item is not None]
    if len(batch) == 0:
        return None
    return torch.utils.data.dataloader.default_collate(batch)


In [20]:
transform = tio.Compose([
    tio.RescaleIntensity(out_min_max=(0, 1)), 
    tio.Resize((256, 256, 256)),  # ver como podemos optimizar haciendo resize a los .nii.gz de OASIS de por si son 256X256X256 pero podriamos pobrar talvez con un 96X96X96 o 128X128X128
])

In [ ]:
df = pd.read_csv('DATA_PATH.csv')
train_data = df.sample(frac=0.6, random_state=42)
val_data = df.drop(train_data.index).sample(frac=0.5, random_state=42)
test_data = df.drop(train_data.index).drop(val_data.index)
train_data.to_csv('train.csv', index=False)
val_data.to_csv('val.csv', index=False)
test_data.to_csv('test.csv', index=False)

train_dataset = AlzheimerDataset('train.csv', transform=transform)
val_dataset = AlzheimerDataset('val.csv', transform=transform)
test_dataset = AlzheimerDataset('test.csv', transform=transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    collate_fn=custom_collate
)
val_loader = DataLoader(
    val_dataset,
    batch_size=2,
    shuffle=False,
    num_workers=0,
    collate_fn=custom_collate
)
test_loader = DataLoader(
    test_dataset,
    batch_size=2,
    shuffle=False,
    num_workers=0,
    collate_fn=custom_collate
)


# Modelo

Cargamos la ResNet18 con los pesos preentrenados de MedicalNet.

In [22]:
model = resnet18(
    sample_input_D=256,
    sample_input_H=256,
    sample_input_W=256,
    num_classes=1,
    pretrained_weights='pretrain/resnet_18_23dataset.pth',  
    freeze_conv_layers=True
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

c:\Users\nicol\Documents\gliara\models\resnet_alzheimer.py:193: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_dict = torch.load(pretrained_weights, map_location='

Loaded pre-trained weights from pretrain/resnet_18_23dataset.pth. Mismatched layers were skipped.


In [23]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

### Entrenamiento

In [ ]:
num_epochs = 10
best_loss = float('inf')
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
    for imgs, labels in train_bar:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)

        train_bar.set_postfix(loss=loss.item())

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

    if epoch_loss < best_loss:
        best_loss = epoch_loss
        torch.save(model.state_dict(), 'best_model.pth')
        print(f'Saved best model with loss: {best_loss:.4f}')


---

## DESPUES PARA PROBAR EL MODELO POR UNO

In [ ]:
#    model.eval()
#    img_path = 'ruta'
#    try:
#        img = nib.load(img_path).get_fdata()
#        img = torch.tensor(img, dtype=torch.float32).unsqueeze(0)
#        img = transform(img).unsqueeze(0).to(device)
#        with torch.no_grad():
#            output = model(img).squeeze()
#            prob = torch.sigmoid(output).item()
#            pred = 1 if prob > 0.5 else 0
#            print(f'Probability of No Alzheimer’s: {prob:.4f}')
#            print(f'Prediction: {"No Alzheimer’s" if pred == 1 else "Alzheimer’s"}')
#    except FileNotFoundError:
#        print(f"File not found: {img_path}")